# Nightlights Generator

---
*Christian Johannes Meyer ([christian.meyer@eui.eu](christian.meyer@eui.eu)) and Espen Beer Prydz ([eprydz@worldbank.org](eprydz@worldbank.org))*
 
Sept 2, 2016

---

This notebook proccesses the CSV file generated using [Google Earth Engine](https://earthengine.google.com) and saved in Google Drive.

---

## 1. Preliminary

In [1]:
%matplotlib inline

#### Import relevant packages

In [2]:
# Pandas
from pandas import Series, DataFrame
import pandas as pd

# Pandas datareader for WDI API access
from pandas_datareader import wb as wdi

# Other packages we may need
import os, fnmatch, sys, datetime, time, itertools, fiona

#### Parameters

In [3]:
# Define paths to input and output files
CSV_PATH = 'output' 
GEOMETRY_PATH = 'static' 

# Imported files
EEDATA_FILE = 'OpenEarth_Nightlights_GEEexport.csv'
EEMETADATA_FILE = 'wb_geo_list.csv' # metadata for WB geometries

# Exported files
OUTPUTWBGEO_FILE = 'OpenEarth_Nightlights_byWBGeometry.csv'
OUTPUTISO_FILE = 'OpenEarth_Nightlights_byWBiso3c.csv'

# Source collection or image
SOURCE = 'NOAA/DMSP-OLS/_NIGHTTIME_LIGHTS'

# Settings for EE data
# This should be changed in production
REDUCTION_SCALE = 1000

## 2. Get EE Data and Merge with Geometry Metadata

Read in CSV files with EE data and geometry metadata

In [58]:
ee_data_df = pd.read_csv(CSV_PATH + '/' + EEDATA_FILE)
ee_meta_df = pd.read_csv(GEOMETRY_PATH + '/' + EEMETADATA_FILE)

In [59]:
ee_data_df['sat'] = ee_data_df['system:index'].str[:3]
ee_data_df['year'] = ee_data_df['system:index'].str[-4:]
ee_data_df['ee_source'] = SOURCE
ee_data_df['ee_reductionscale'] = REDUCTION_SCALE
ee_data_df = ee_data_df.drop('.geo', 1)
ee_data_df = ee_data_df.drop('time', 1)
# df['date'] = map(lambda t: datetime.datetime.utcfromtimestamp(t/1000), df['time'])

Wrangle EE data into better shape

In [61]:
# Reshape
ee_data_df = pd.wide_to_long(ee_data_df, 'j', i='system:index',j='wbgeometry_uid')
ee_data_df = ee_data_df.reset_index(level=['wbgeometry_uid'])
# Clean up column names
ee_data_df = ee_data_df.rename(columns={'j': 'ee_nightlights_sum'})

Merge data extracted from Google Earth Engine with WB geometry metadata

In [173]:
ee_df = pd.merge(ee_data_df,ee_meta_df, how='outer', on='wbgeometry_uid')

Set correct hierarchical index and export to CSV

In [174]:
ee_df.set_index(['wbgeometry_uid','year','sat']).to_csv(CSV_PATH + '/' + OUTPUTWBGEO_FILE, sep=',')

## 3. Get Basic Indicators and Metadata from WDI

Extract country metadata from WDI

In [64]:
wdi_meta_df = wdi.get_countries()
wdi_meta_df.rename(columns = {'name':'country'}, inplace = True)

Query a few indicators from WDI

In [191]:
wdi_indicators = ['SP.POP.TOTL', 'NY.GDP.PCAP.CD', 'NY.GNP.PCAP.CD', 'NY.GNP.PCAP.PP.CD','NY.GDP.PCAP.KD','NY.GDP.PCAP.PP.KD']
wdi_data_df = wdi.download(indicator=wdi_indicators, country='all', start=2000, end=2013).dropna().reset_index()

Merge data extracted from WDI API with WBI metadata

In [192]:
wdi_df = pd.merge(wdi_data_df,wdi_meta_df, how='outer', on='country')

## 4. Merge EE data into WDI data

We first need to aggregate night lights data from EE by iso3c, since some countries have several geometries in the WB GeoJSON file. We sum up lights by country.

In [179]:
ee_df_grouped = ee_df.set_index(['iso3c','wbgeometry_uid','year','sat'])
ee_df_grouped.sort_index(inplace=True)
ee_df_grouped = ee_df_grouped.sum(level=['iso3c','year','sat'])[['ee_nightlights_sum']].reset_index()

#### Merge grouped EE data with WDI data. 
For illustrative purposes, we use the intersection of keys from both frames (as opposed to keys from only WDI, only from EE, or the union).

In [202]:
df = pd.merge(wdi_df, ee_df_grouped, how='inner', on=['iso3c','year'])

Export to CSV

In [203]:
df.to_csv(CSV_PATH + '/' + OUTPUTISO_FILE, sep=',', encoding='utf-8')